# Finding scope 3 emissions of a Pharma company.
- Scope 3 category is downstream transportation and distribution
- Data taken from data.world
- Distance-based methology will be used for calculations
    - According to GHG "If sub-contractor fuel data cannot be easily obtained in order to use the fuel-based method, then the distance-based method should be used."

# Data points required for calculations of scope 3
1. Total distance in KM 
    - Coordinates of places
2. Size of packages
3. Mode of Transport
4. Emission Factors
5. Calculation formulaes
    1. Emissions from road transport: = ∑ (mass of goods purchased (tonnes) × distance travelled in transport leg × emission factor of transport mode or vehicle type (kg CO2e/tonne-km)
    2. emissions from air transport: = ∑ (quantity of goods purchased (tonnes) x distance travelled in transport leg x emission factor of transport mode or vehicle type (kg CO2e/tonne-km))
    3. emissions from sea transport: = ∑ (quantity of goods purchased (tonnes) x distance travelled in transport leg x emission factor of transport mode or vehicle type (kg CO2e/tonne-km))

### Steps I took
1. Searching for dataset
2. Understanding the dataset and making calculative assumptions
    - Assumptions made should have proper justification.
3. Listing down the data points required for calculations
4. Listing down the calculation formulaes
5. Filtering out the data columns required from main data
6. Exploratory Data Analysis
7. Choosing a single year for calculations
8. Data cleaning of Weight column and Mode column.
9. Got all airport and ports data to find the nearest transport location to the manufacturing site.
10. Merged the source location df and main df


## Data columns
1. Country - Destination location
2. Mode - Mode of transport (Air, Ocean, Road)
3. Weight - Kilogram of weight
4. Delivery Date - Date of delivery
5. Manufacturing Site - Source location

In [60]:
# Import libraries
import pandas as pd
import numpy as np
import geopandas as gpd

# For geolocation
from geopy.geocoders import ArcGIS

# For calculation of source airport/port
from math import radians, sin, cos, sqrt, atan2

# Calculating the time
import time

# For converting country name to iso_2 codes
import country_converter as coco

In [2]:
# Collecting data
raw_df = pd.read_csv("./Data/SCMS_Delivery_History_Dataset_20150929.csv")

In [3]:
# Filtering out required columns from the main data.

filtered_df = raw_df[['ID','Country','Shipment Mode','Manufacturing Site','Weight (Kilograms)','Delivery Recorded Date','Item Description']].copy()
filtered_df.rename(columns={"Shipment Mode":"Mode","Weight (Kilograms)":"Weight","Delivery Recorded Date":"Delivery_Date",'Manufacturing Site':'Manufacturing_site','Country':'Destination_Country'},inplace=True)

In [4]:
# Converting Delivery Date to datetime datatype
filtered_df.loc[:,'Delivery_Date'] = pd.to_datetime(filtered_df['Delivery_Date'])

# Exploratory Data Analysis

In [5]:
# Checking total number of data points for each year
filtered_df['Delivery_Date'].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      65
2007     670
2008    1109
2009    1192
2010    1176
2011    1049
2012    1250
2013    1205
2014    1599
2015    1009
Name: Delivery_Date, dtype: int64

In [6]:
# Check for null values
filtered_df.isna().sum()

ID                       0
Destination_Country      0
Mode                   360
Manufacturing_site       0
Weight                   0
Delivery_Date            0
Item Description         0
dtype: int64

Null values found in the mode of transport. 

In [7]:
# Checking Mode column's null values

# filtered_df[filtered_df['Mode'].isna()]

# Counting the number of null values per year
filtered_df[filtered_df['Mode'].isna()].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      2
2007    264
2008     94
dtype: int64

Since mode of transport is not available for these years (2006,2007,2008) I will not consider these years for our base year.
Even though year 2006 has only 2 NaN rows for Mode, the number of data points are only 65. Thus not enough to consider as base year as compared to other years.

In [8]:
# Checking Weight column values
filtered_df[filtered_df['Weight'].str.isnumeric() == False]

,ID,Destination_Country,Mode,Manufacturing_site,Weight,Delivery_Date,Item Description
8,46,Nigeria,Air,"Aurobindo Unit III, India",See ASN-93 (ID#:1281),2006-12-07,"Stavudine 30mg, capsules, 60 Caps"
12,62,Nigeria,Air,"EY Laboratories, USA",Weight Captured Separately,2007-01-10,"HIV 1/2, InstantChek HIV 1+2 Kit, 100 Tests"
15,68,Zimbabwe,Air,"BMS Meymac, France",Weight Captured Separately,2007-03-19,"#102198**Didanosine 200mg [Videx], tablets, 60..."
16,69,Nigeria,NaN,ABBVIE GmbH & Co.KG Wiesbaden,Weight Captured Separately,2007-05-07,"HIV 1/2, Determine Complete HIV Kit, 100 Tests"
31,262,South Africa,NaN,GSK Mississauga (Canada),Weight Captured Separately,2008-01-29,"Zidovudine 10mg/ml [Retrovir], oral solution, ..."
...,...,...,...,...,...,...,...
10318,86817,Zimbabwe,Truck,"Cipla, Goa, India",See DN-4307 (ID#:83920),2015-07-20,"Lamivudine/Nevirapine/Zidovudine 30/50/60mg, d..."
10319,86818,Zimbabwe,Truck,"Mylan, H-12 & H-13, India",See DN-4307 (ID#:83920),2015-07-20,"Lamivudine/Nevirapine/Zidovudine 30/50/60mg, d..."
10320,86819,C�te d'Ivoire,Truck,Hetero Unit III Hyderabad IN,See DN-4313 (ID#:83921),2015-08-07,"Lamivudine/Zidovudine 150/300mg, tablets, 60 Tabs"
10321,86821,Zambia,Truck,Cipla Ltd A-42 MIDC Mahar. IN,Weight Captured Separately,2015-09-03,Efavirenz/Lamivudine/Tenofovir Disoproxil Fuma...


As you can see above the weight column has non-numeric values 'Weight Captured Separately', thus the values for these rows are not available. While values like 'See ASN-93 (ID#:1281)' can be found by mapping to that particular ID and getting the weight

In [9]:
# Counting the number of 'Weight Captured Separately' values in weight columns per year.

filtered_df[(filtered_df['Weight'].str.isnumeric() == False) & (filtered_df['Weight'] == 'Weight Captured Separately')].groupby(by=filtered_df['Delivery_Date'].dt.year).size()

Delivery_Date
2006      6
2007     28
2008    233
2009    262
2010     67
2011     56
2012     33
2013     80
2014    386
2015    356
dtype: int64

Seeing that 2012 has least number of 'Weight Captured Separately' values and it has good number of data points i.e. 1250. We can consider it as our base year.

# Data Cleaning

## Fixing Weight column

In [10]:
# Filtering out 'Weight Captured Separately' rows from Final dataset
final_df = filtered_df[(filtered_df['Delivery_Date'].dt.year == 2012) & (filtered_df['Weight'] != 'Weight Captured Separately')].copy()
final_df.reset_index(inplace = True, drop = True)

In [11]:
# Function to resolve "See DN-2947 (ID#:83642)" given in weight column and adding the weight

def mapWeights(weight):
    try:
        if weight.isnumeric() == False:
            ID = weight[-6:-1]
            weight_returned = filtered_df[filtered_df['ID'] == int(ID)]['Weight'].iloc[0]
            if weight_returned == 'Weight Captured Separately':
                return None
#             print(f'{ID} -- {weight_returned}')
            return float(weight_returned)
        return float(weight)
    except Exception as e:
        print(f'Error == {e} \n {weight[-6:-1]} --- {filtered_df[filtered_df["ID"] == int(weight[-6:-1])]["Weight"].iloc[0]}')

In [12]:
weights = final_df['Weight'].apply(mapWeights)

In [13]:
# Adding weights list to the df
final_df.loc[:,'Weight'] = weights

In [14]:
# Removing None value rows for weight

final_df = final_df[final_df['Weight'].isna() == False]

In [15]:
# Converting Weights from KG to Tonnes
# 1 kilogram = 0.001 tonne

final_df['Weight'] = final_df['Weight'] * 0.001

## Fixing Mode column

In [16]:
final_df['Mode'].unique()

array(['Air', 'Ocean', 'Truck', 'Air Charter'], dtype=object)

As you can see we have air charter as the 4th Mode of transport. We can combine that with Air transport

In [17]:
final_df.loc[final_df['Mode'] == 'Air Charter','Mode'] = 'Air'

## Getting Location coordinates of Source Location and Destination Location

## To-Do
1. Source 
- If the mode of transport is 
        1. Air and Ocean, then we get the Airport/Port coordinates of the nearest Airport/Port to the manufacturing site.
        2. Truck, then we get the coordinates of the manufacturing site.
2. Destination
- If the mode of transport is 
        1. Air and Truck, then we get the Airport coordinates of the Capital City of the Country given.
        2. Ocean, then we get the coordinates of the main port of the country.

In [38]:
# Read airports data
raw_airport_df = pd.read_csv('Data/airports.csv')
# Read ports data
raw_ports_df = gpd.read_file('Data/attributed_ports.geojson')

In [51]:
# Data cleaning for airports and ports data
df = raw_airport_df[['type','latitude_deg','longitude_deg','name','iso_country','municipality']]
final_airport_df = df[df['type'].str.lower().str.contains('airport')].copy()

final_ports_df = raw_ports_df[['Country','Name','geometry']].copy()
final_ports_df['latitude_deg'] = final_ports_df.geometry.apply(lambda p: p.y)
final_ports_df['longitude_deg'] = final_ports_df.geometry.apply(lambda p: p.x)
final_ports_df.drop(['geometry'], axis=1,inplace=True)
final_ports_df['iso_country'] = coco.convert(final_ports_df['Country'],to='ISO2', not_found=None)

### Finding source location coordinates

In [39]:
# Initialising geolocation object
nom = ArcGIS()
# DF to append Source location
data = {'Manufacturing_site':[],'latitude':[],'longitude':[],'Mode':[]}
cc = coco.CountryConverter()
source_locations = pd.DataFrame(data)

In [22]:
# Functions to get the distance in KM
def distance(lat1, lon1, lat2, lon2):
    # Calculate the distance between two coordinates using the haversine formula
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [23]:
"""
Function to check 
Check in source location DF if the place and mode are already calculated.
If not calculated, it means that for this place and mode the source has not been calculated.
If already calculated, then skip to next manufacturing site
"""
def checkSourceLocation(place,mode):
    if source_locations.size == 0:
        return True
    for index,row in source_locations.iterrows():
           if row['Manufacturing_site'] == place and row['Mode'] == mode:
                   return False
    return True

In [24]:
"""
Function to find the minimum distance
"""
def minDistance(transport_df,location,iso2_country):
    min_distance = float('inf')
    min_latitude = float('inf')
    min_longitude = float('inf')

    # Getting all airports lying inside the country
    transport_df_country = transport_df[transport_df['iso_country'] == iso2_country]

    for index,row in transport_df_country.iterrows():
        dist = distance(location.latitude, location.longitude, row['latitude_deg'],row['longitude_deg'])
        if dist <= min_distance:
            min_distance = dist
            min_latitude = row['latitude_deg']
            min_longitude = row['longitude_deg']

    return min_latitude,min_longitude

In [25]:
# Function to get the coordinates of the location
# Now that we have location of the manufacturing site coordinates. We need to get the Airport/Port location from which the item was transported
def getSourceLocation(data):
    # Code to Measure time taken by program to execute.
    # store starting time
    begin = time.time()
#   print(data)
    place = data['Manufacturing_site']
    mode = data['Mode']
    try:
        
        # Check if source location has already been calculated or not
        if checkSourceLocation(place,mode):
            location = nom.geocode(place)
            country = nom.reverse(str(location.latitude) + ',' + str(location.longitude)).address.split(',')[-1].replace(" ", "")
            # Getting iso_2 code for the country
            iso2_country = cc.convert(names=country, to='ISO2',not_found=None)
      
            # Now that we have location of the manufacturing site. We need to get the Airport/Port location from which the item was transported
            if mode == 'Air':
                # Finding the airport which has minimum distance
                min_latitude,min_longitude = minDistance(final_airport_df,location,iso2_country)
                source_locations.loc[len(source_locations.index)] = [place,min_latitude,min_longitude,mode]
            elif mode == 'Ocean':
                # Finding the port which has minimum distance
                min_latitude,min_longitude = minDistance(final_ports_df,location,iso2_country)
                source_locations.loc[len(source_locations.index)] = [place,min_latitude,min_longitude,mode]
            else:
                source_locations.loc[len(source_locations.index)] = [place,location.latitude,location.longitude,mode]
        else:
            print(f'Already Completed {place} {mode}')
    except Exception as e:
        print(e, place)
        source_locations.loc[len(source_locations.index)] = [place,None,None,mode]

In [26]:
# Adding source_latitude and source_longitude columns
# store starting time
begin = time.time()
final_df.apply(getSourceLocation,axis=1)
end = time.time()
# total time taken
print(f"Total runtime of the program is {end - begin}")

Already Completed Alere Medical Co., Ltd. Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Trinity Biotech, Plc Air
Already Completed Trinity Biotech, Plc Air
Already Completed Chembio Diagnostics Sys. Inc. Air


2643 not found in ISOnumeric


Already Completed Aurobindo Unit III, India Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Trinity Biotech, Plc Air
Already Completed Aurobindo Unit III, India Air
Already Completed Aurobindo Unit III, India Air
Already Completed Mylan (formerly Matrix) Nashik Air
Already Completed Chembio Diagnostics Sys. Inc. Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Trinity Biotech, Plc Air
Already Completed Hetero Unit III Hyderabad IN Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Aurobindo Unit III, India Air
Already Completed KHB Test Kit Facility, Shanghai China Air
Already Completed Mylan (formerly Matrix) Nashik Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Cipla, Goa, India Air
Already Completed Inverness Japan Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Chembio Diagnostics Sys. Inc. Air
Already Completed Mylan (formerly Matrix) Nashik Air
Already Completed Alere Medical Co., Ltd

충청북도옥천군청산면교평리 not found in regex


Already Completed Aurobindo Unit III, India Air
Already Completed Aspen-OSD, Port Elizabeth, SA Air
Already Completed Mylan (formerly Matrix) Nashik Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Chembio Diagnostics Sys. Inc. Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed Mylan (formerly Matrix) Nashik Air
Already Completed Trinity Biotech, Plc Air
Already Completed ABBVIE (Abbott) Logis. UK Air
Already Completed MSD, Haarlem, NL Air
Already Completed Mylan (formerly Matrix) Nashik Air
Already Completed Mylan (formerly Matrix) Nashik Air
Already Completed Aurobindo Unit III, India Air
Already Completed Alere Medical Co., Ltd. Air
Already Completed MSD, Haarlem, NL Air
Already Completed Hetero Unit III Hyderabad IN Air
Already Completed MSD, Haarlem, NL Air
Already Completed Cipla, Goa, India Air
Already Completed Trinity Biotech, Plc Air
Already Completed Alere Medical Co., Ltd. Air
Already Complete

The code took around 74 seconds to complete the computations

Merging the source_location_df to the final_df

In [27]:
final_df = final_df.merge(source_locations, how='outer', on=['Manufacturing_site','Mode'])

In [28]:
final_df

,ID,Destination_Country,Mode,Manufacturing_site,Weight,Delivery_Date,Item Description,latitude,longitude
0,12973,Haiti,Air,"Cipla, Goa, India",0.021,2012-06-12,"Lamivudine/Nevirapine/Stavudine 30/50/6mg, dis...",15.380800,73.831398
1,19105,Rwanda,Air,"Cipla, Goa, India",1.631,2012-06-22,"Abacavir 20mg/ml, oral solution, Bottle, 240 ml",15.380800,73.831398
2,39441,Zambia,Air,"Cipla, Goa, India",5.999,2012-06-26,"Lamivudine/Nevirapine/Stavudine 60/100/12mg, d...",15.380800,73.831398
3,46005,Rwanda,Air,"Cipla, Goa, India",6.490,2012-06-28,"Lamivudine 10mg/ml, oral solution, Bottle, 240 ml",15.380800,73.831398
4,46574,Rwanda,Air,"Cipla, Goa, India",0.049,2012-06-15,"Lamivudine/Stavudine 60/12mg, dispersible tabl...",15.380800,73.831398
...,...,...,...,...,...,...,...,...,...
1197,84152,South Sudan,Air,Macleods Daman Plant INDIA,2.652,2012-03-08,"Lamivudine 150mg, tablets, 60 Tabs",20.434401,72.843201
1198,84166,Libya,Air,Macleods Daman Plant INDIA,0.709,2012-05-23,"Lamivudine 150mg, tablets, 60 Tabs",20.434401,72.843201
1199,85257,Dominican Republic,Air,Macleods Daman Plant INDIA,0.544,2012-05-04,"Lamivudine 150mg, tablets, 60 Tabs",20.434401,72.843201
1200,85866,"Congo, DRC",Air,Macleods Daman Plant INDIA,0.895,2012-09-11,"Lamivudine 150mg, tablets, 60 Tabs",20.434401,72.843201


Checking for any NONE or INF values in longitude and latitude columns

In [29]:
final_df.loc[(final_df['latitude'].isna()) | (final_df['latitude'] == float('inf'))]

,ID,Destination_Country,Mode,Manufacturing_site,Weight,Delivery_Date,Item Description,latitude,longitude
669,16085,Haiti,Air,"Medochemie Factory A, CY",1.173,2012-01-18,"Mefloquine 250mg (as hydrochloride), tablets, ...",inf,inf
745,24246,Ethiopia,Air,ABBVIE (Abbott) Logis. UK,1.050,2012-06-27,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
746,26019,Rwanda,Air,ABBVIE (Abbott) Logis. UK,0.253,2012-06-29,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
747,29677,Vietnam,Air,ABBVIE (Abbott) Logis. UK,0.275,2012-11-29,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
748,37960,Zimbabwe,Air,ABBVIE (Abbott) Logis. UK,1.374,2012-12-13,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
749,37979,Zimbabwe,Air,ABBVIE (Abbott) Logis. UK,1.374,2012-11-08,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
750,44577,Zimbabwe,Air,ABBVIE (Abbott) Logis. UK,1.014,2012-12-13,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
751,51217,Zimbabwe,Air,ABBVIE (Abbott) Logis. UK,1.254,2012-04-02,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
752,65987,Vietnam,Air,ABBVIE (Abbott) Logis. UK,0.164,2012-06-27,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN
753,68268,Ethiopia,Air,ABBVIE (Abbott) Logis. UK,0.728,2012-02-20,"Lopinavir/Ritonavir 80/20mg/ml [Kaletra], oral...",NaN,NaN


In [30]:
indexes = final_df.loc[(final_df['latitude'].isna()) | (final_df['latitude'] == float('inf'))].index

Since coordinates of these manufacturing sites were not found, I will drop these.

In [31]:
final_df.drop(index=indexes,axis=1,inplace=True)

In [32]:
final_df.loc[(final_df['latitude'].isna()) | (final_df['latitude'] == float('inf'))]

,ID,Destination_Country,Mode,Manufacturing_site,Weight,Delivery_Date,Item Description,latitude,longitude


Adding suffixes to latitude and longitude columns

In [33]:
final_df.rename(columns={'latitude':'source_lat','longitude':'source_long','Manufacturing_site':'Source_site'},inplace=True)

In [36]:
final_df.head(1)

,ID,Destination_Country,Mode,Source_site,Weight,Delivery_Date,Item Description,source_lat,source_long
0,12973,Haiti,Air,"Cipla, Goa, India",0.021,2012-06-12,"Lamivudine/Nevirapine/Stavudine 30/50/6mg, dis...",15.3808,73.831398


### Getting Destination location coordinates

## TO-DO:
Destination
- If the mode of transport is 
        1. Air and Truck, then we get the Airport coordinates of the Capital City of the Country given.
        2. Ocean, then we get the coordinates of the main port of the country.

In [42]:
# Getting ISO_2 codes for each country
final_df['destination_iso_country'] = coco.convert(final_df['Destination_Country'],to='ISO2', not_found=None)

Uploading country capital list and cleaning the data

In [58]:
country_capital = pd.read_csv('Data/country-capital-list.csv')
country_capital.drop(columns=['type'],axis=1,inplace=True)
country_capital['iso_country'] = coco.convert(country_capital['country'],to='ISO2', not_found=None)

Abkhazia not found in regex
Akrotiri and Dhekelia not found in regex
Ascension Island not found in regex
Easter Island not found in regex
Nagorno-Karabakh Republic not found in regex
Scotland not found in regex
South Ossetia not found in regex
Transnistria not found in regex
Tristan da Cunha not found in regex
Wales not found in regex


Merging the main_df and country capital df to get the required capital cities.

In [108]:
destination_locations = country_capital.merge(final_df[['Mode','destination_iso_country']],right_on='destination_iso_country',left_on='iso_country').drop(columns=['destination_iso_country'])

In [109]:
destination_locations

,country,capital,iso_country,Mode
0,Botswana,Gaborone,BW,Air
1,Botswana,Gaborone,BW,Air
2,Burundi,Bujumbura,BI,Air
3,Burundi,Bujumbura,BI,Air
4,Burundi,Bujumbura,BI,Air
...,...,...,...,...
1184,Zimbabwe,Harare,ZW,Truck
1185,Zimbabwe,Harare,ZW,Truck
1186,Zimbabwe,Harare,ZW,Truck
1187,Zimbabwe,Harare,ZW,Truck


Dropping duplicate rows

In [112]:
destination_locations.drop_duplicates(inplace=True)

In [114]:
destination_locations

,country,capital,iso_country,Mode
0,Botswana,Gaborone,BW,Air
2,Burundi,Bujumbura,BI,Air
17,Cameroon,Yaoundé,CM,Air
31,Côte d'Ivoire,Yamoussoukro,CI,Air
71,Côte d'Ivoire,Yamoussoukro,CI,Ocean
81,Côte d'Ivoire,Yamoussoukro,CI,Truck
145,Democratic Republic of the Congo,Kinshasa,CD,Air
235,Dominican Republic,Santo Domingo,DO,Air
241,Ethiopia,Addis Ababa,ET,Air
266,Ghana,Accra,GH,Air


In [115]:
final_airport_df[final_airport_df['municipality'].str.lower() == 'gaborone']

,type,latitude_deg,longitude_deg,name,iso_country,municipality
24003,large_airport,-24.555201,25.9182,Sir Seretse Khama International Airport,BW,Gaborone
